## Assignment 1

In [63]:
import numpy as np
from datasets import load_dataset,concatenate_datasets

### Loading The Data

In [64]:
dataset = load_dataset('sst', trust_remote_code=True)

In [65]:
dataset['test'][10]

{'sentence': 'What really surprises about Wisegirls is its low-key quality and genuine tenderness .',
 'label': 0.625,
 'tokens': 'What|really|surprises|about|Wisegirls|is|its|low-key|quality|and|genuine|tenderness|.',
 'tree': '25|24|22|21|21|19|16|15|15|17|14|14|20|18|16|17|18|19|20|23|22|23|24|25|0'}

### Preprocessing

In [66]:
def map_class(value):
    return np.digitize(value, bins=[0.2, 0.4, 0.6, 0.8], right=False)

In [67]:
dataset_mapped = dataset.map(lambda example: {'label': map_class(example['label'])})

In [68]:
# Access the training set
train_data = concatenate_datasets([dataset_mapped['train'], dataset_mapped['validation']])
train_labels = np.array(train_data['label'],dtype=np.int8)

# Access the test set
test_data = dataset_mapped['test']
test_labels = np.array(test_data['label'],dtype=np.int8)

num_classes = 5

In [69]:
x = test_data[10]
x

{'sentence': 'What really surprises about Wisegirls is its low-key quality and genuine tenderness .',
 'label': 3.0,
 'tokens': 'What|really|surprises|about|Wisegirls|is|its|low-key|quality|and|genuine|tenderness|.',
 'tree': '25|24|22|21|21|19|16|15|15|17|14|14|20|18|16|17|18|19|20|23|22|23|24|25|0'}

### Part 1: Naïve Bayes 

#### Algorithm Implementation

In [70]:
def train_naive_bayes(D,num_classes):
     
    n_doc = len(D)  # Total number of documents/examples
    cls_prior = np.zeros(num_classes)  # Initialize class prior probabilities
    vocab = set()  # To store the vocabulary
    cls_word_cnt = np.array([{} for _ in range(num_classes)])  # Initialize class word counts

    # Loop through each example in the dataset
    for example in D:
        example_class = int(example["label"])  # Get the class label for the example
        cls_prior[example_class] += 1  # Count the number of documents for each class
        for word in example["tokens"].split("|"):  # Tokenize the example
            # Update word count for the corresponding class
            cls_word_cnt[example_class][word] = cls_word_cnt[example_class].get(word, 0) + 1
            vocab.add(word)  # Add word to the vocabulary
           
    cls_prior /= n_doc  # Calculate prior probabilities
    log_prior = np.log(cls_prior)  # Convert prior probabilities to log space
    
    vocab_size = len(vocab)
    cls_total_word_cnt = np.array([sum(cls_dict.values()) for cls_dict in cls_word_cnt])  # Total word count for each class
    
    # Calculate log likelihood with smoothing (Laplace smoothing)
    log_likelihood = np.array([
        {word: np.log((cls_dict.get(word,0) + 1) / (cls_total_word_cnt[i] + vocab_size)) for word in vocab} 
        for i, cls_dict in enumerate(cls_word_cnt)
    ])
    
    return log_prior, log_likelihood, vocab

def test_naive_bayes(test_doc,log_prior, log_likelihood,num_classes,vocab):
    sum_lg = np.zeros(num_classes)
    for c in range(num_classes):
        sum_lg[c] = log_prior[c]
        for word in test_doc.split():
            if word in vocab:
                sum_lg[c] += log_likelihood[c][word]
                
    return np.argmax(sum_lg)

In [71]:
log_prior, log_likelihood, vocab = train_naive_bayes(train_data,num_classes)

In [72]:
predictions = []
for doc in test_data:
    predictions.append(test_naive_bayes(doc["sentence"],log_prior, log_likelihood, num_classes, vocab))
    
predictions = np.array(predictions)

# Calculate the number of correct predictions
correct_predictions = np.sum(predictions == test_labels)
    
# Calculate accuracy
my_accuracy = correct_predictions / len(test_labels)

In [73]:
print(my_accuracy)

0.3995475113122172


#### Comparison with scikit learn

In [74]:
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report


# Define the pipeline
pipeline = Pipeline([
    ('vectorizer', CountVectorizer()), 
    ('classifier', MultinomialNB())
])

# Fit the pipeline to the training data
pipeline.fit(train_data["sentence"], train_labels)

# Make predictions on the test data
y_pred = pipeline.predict(test_data["sentence"])

# Evaluate
print("Accuracy:", accuracy_score(test_labels, y_pred))

Accuracy: 0.40497737556561086


### Part 2: Logistic Regression

#### Feature Representation

In [75]:
def get_bigrams_cnt(D):
    
    bigrams = {}
    for example in D:
        words_list = example["tokens"].split("|")
        for i in range(len(words_list)-1):
            bigram = words_list[i] + " " + words_list[i+1]
            if bigram not in bigrams:
                bigrams[bigram] = len(bigrams)
    
    return bigrams
                
def generate_feat(D,bigrams,num_classes):
    test_data_feat = np.zeros((len(D),len(bigrams)+1),dtype=np.int8)
    test_data_labels = np.zeros((len(D),num_classes),dtype=np.int8)
    for i,example in enumerate(D):
        test_data_labels[i][int(example["label"])] = 1
        words_list = example["tokens"].split("|")
        for j in range(len(words_list)-1):
            bigram = words_list[j] + " " + words_list[j+1]
            if bigram in bigrams:
                test_data_feat[i][bigrams[bigram]] = 1
            
        
    return test_data_feat ,test_data_labels     


In [76]:
bigrams = get_bigrams_cnt(train_data)
X,Y = generate_feat(train_data,bigrams,num_classes)
print(Y.shape)
print(len(X))
print(Y)

(9645, 5)
9645
[[0 0 0 1 0]
 [0 0 0 0 1]
 [0 0 0 1 0]
 ...
 [0 1 0 0 0]
 [0 0 1 0 0]
 [0 1 0 0 0]]


#### Algorithm Implementation

In [115]:
def create_batches(X, Y, batch_size):
    for i in range(0, len(X), batch_size):
        batch_X = X[i:min(i + batch_size,len(X))]
        batch_Y = Y[i:min(i + batch_size,len(X))]
        yield batch_X, batch_Y

def softmax(X:np.ndarray,W:np.ndarray,B:np.ndarray):  
    W_exp = np.exp(X @ W + B )   #N*f x f*c = N*c
    W_exp_sum = np.sum(W_exp,axis = 1).reshape(-1,1)

    return W_exp/W_exp_sum

def loss(X:np.ndarray,Y:np.ndarray,W:np.ndarray,B:np.ndarray):
    probabilities = softmax(X, W, B)
    log_likelihood = -np.log(np.sum(Y * probabilities, axis=1))
    return np.mean(log_likelihood)
    
def calculate_gradient(X:np.ndarray,Y:np.ndarray,W:np.ndarray,B:np.ndarray):
    p = Y - softmax(X, W, B)
    dW = - (X.T @ p) / X.shape[0]
    dB = - np.sum(p, axis=0, keepdims=True) / X.shape[0]
    
    return  dW,dB
          
def logistic_regression(X: np.ndarray, Y: np.ndarray, num_classes, 
                        lr=0.001, max_itr=100, batch_size=15, 
                        decay_factor=1, step_size=100,loaded_W = None,loaded_B = None):
    
    _, num_features = X.shape
    
    W = np.zeros((num_features,num_classes))
    B = np.zeros((1,num_classes))
    if loaded_W is not None and loaded_B is not None:
        W = loaded_W
        B = loaded_B
 
    # Initialize variables to store the best loss and model parameters
    best_loss = float('inf')
    W_best = None
    B_best = None
    itr = 0
    
    for itr in range(max_itr):
        print(f"Iteration {itr+1}:")
        print("=============================================================================")
        
        epoch_loss = 0  # Initialize the loss for the entire epoch
        num_batches = 0  # Counter for the number of batches processed
        
        for batch_X, batch_Y in create_batches(X, Y, batch_size):
            # Calculate gradients
            dW, dB = calculate_gradient(batch_X, batch_Y, W, B)
            
            # Update weights and biases
            W -= lr * dW
            B -= lr * dB
            
            # Calculate the batch loss
            batch_loss = loss(batch_X, batch_Y, W, B)
            epoch_loss += batch_loss
            num_batches += 1  # Increment batch counter
        
        # Calculate average loss over all batches
        total_loss = epoch_loss / num_batches
        
        # Update the best loss if current epoch loss is lower
        if total_loss < best_loss:
            best_loss = total_loss
            W_best = W.copy()
            B_best = B.copy()
            
        # Step decay of learning rate
        if (itr + 1) % step_size == 0:  # If we've completed a step
            lr *= decay_factor  # Reduce the learning rate
        
        print(f"The current loss for iteration {itr+1} = {total_loss}")
        print("=============================================================================")
    
    print("Best Loss:", best_loss)
    
    return W, B, W_best, B_best

def predict(X,W,B):
    return np.argmax(softmax(X,W,B),axis=1)   

In [108]:
W, B, W_best, B_best = logistic_regression(X,Y,num_classes,lr=0.1,max_itr=100,batch_size=1)

Iteration 1:
The current loss for iteration 1 = 0.4220532272236317
Iteration 2:
The current loss for iteration 2 = 0.28308674918788773
Iteration 3:
The current loss for iteration 3 = 0.21527912156665888
Iteration 4:
The current loss for iteration 4 = 0.1742661083574722
Iteration 5:
The current loss for iteration 5 = 0.14673007498203658
Iteration 6:
The current loss for iteration 6 = 0.12694858431703945
Iteration 7:
The current loss for iteration 7 = 0.11202580287704016
Iteration 8:
The current loss for iteration 8 = 0.10034809263323118
Iteration 9:
The current loss for iteration 9 = 0.09094417335552755
Iteration 10:
The current loss for iteration 10 = 0.08319602820774731
Iteration 11:
The current loss for iteration 11 = 0.07669380615301023
Iteration 12:
The current loss for iteration 12 = 0.07115545376468878
Iteration 13:
The current loss for iteration 13 = 0.0663799531953129
Iteration 14:
The current loss for iteration 14 = 0.06221951522874426
Iteration 15:
The current loss for iterat

In [109]:
X_test,Y_test = generate_feat(test_data,bigrams,num_classes)

In [113]:
pred_labels = predict(X_test,W_best, B_best)

accuracy = np.sum(pred_labels == test_labels)/len(test_labels)
accuracy
# loss(X,Y,best_W,best_B)
# loss(X,Y,W,B)

0.34570135746606334

In [90]:
np.savez("last_model.npz", W=W, B=B)
np.savez("best_model.npz", W=W_best, B=B_best)

##### Load the model

In [123]:
# Load the .npz file
data = np.load("best_model.npz")

# Access the W and B matrices
W = data["W"]
B = data["B"]

pred = predict(X_test,W,B)

np.sum(pred == test_labels)/len(test_labels)

0.3556561085972851

In [ ]:
W, B, W_best, B_best = logistic_regression(X,Y,num_classes,lr=1,max_itr=5,batch_size=1,loaded_W=W,loaded_B=B)

Iteration 1:
The current loss for iteration 1 = 0.2638735078212166
Iteration 2:
The current loss for iteration 2 = 0.24669123679043295
Iteration 3:
The current loss for iteration 3 = 0.2292699186075083
Iteration 4:
The current loss for iteration 4 = 0.21423290831714575
Iteration 5:
The current loss for iteration 5 = 0.20110032997485092
Iteration 6:
The current loss for iteration 6 = 0.18952305003514133
Iteration 7:
The current loss for iteration 7 = 0.1792358037750135
Iteration 8:
The current loss for iteration 8 = 0.17003187309234524
Iteration 9:
The current loss for iteration 9 = 0.1617471501482859
Iteration 10:
The current loss for iteration 10 = 0.15424940534230058
Best Loss: 0.15424940534230058


In [128]:
pred_labels = predict(X_test,W_best, B_best)

accuracy = np.sum(pred_labels == test_labels)/len(test_labels)
accuracy

0.3389140271493213

#### Comparison with scikit learn

In [ ]:
from sklearn.linear_model import LogisticRegression
from sklearn.linear_model import SGDClassifier

# Initialize LogisticRegression
clf_logreg = LogisticRegression(max_iter=100, multi_class='multinomial', solver='lbfgs')
clf_logreg.fit(X, train_labels)  # Assuming X_train, Y_train are your training sets


# Initialize SGDClassifier as logistic regression
clf_sgd = SGDClassifier(loss='log_loss', max_iter=100, tol=1e-3, learning_rate='optimal', penalty='l2')
clf_sgd.fit(X, train_labels)

In [ ]:
y_pred_logreg = clf_logreg.predict(X_test)
y_pred_sgd = clf_sgd.predict(X_test)

accuracy = np.sum(y_pred_logreg == test_labels)/len(test_labels)
print(f"logisitc regression accuracy = {accuracy}")

accuracy = np.sum(y_pred_sgd == test_labels)/len(test_labels)
print(f"SGD accuracy = {accuracy}")